# Game of Life Prompt
Implement [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) in 64-bit signed integer space.

Imagine a 2D grid - each cell (coordinate) can be either "alive" or "dead". Every generation of the simulation, the system ticks forward. Each cell's value changes according to the following:

If an "alive" cell had less than 2 or more than 3 alive neighbors (in any of the 8 surrounding cells), it becomes dead.
If a "dead" cell had *exactly* 3 alive neighbors, it becomes alive.
Your input is a list of integer coordinates for live cells in the Life 1.06 format. They could be anywhere in the signed 64-bit range. This means the board could be very large!

Sample input:
```
#Life 1.06

0 1

1 2

2 0

2 1

2 2

-2000000000000 -2000000000000

-2000000000001 -2000000000001

-2000000000000 -2000000000001
```

Your program should read the state of the simulation from standard input, run 10 iterations of the Game of Life, and print the result to standard output in Life 1.06 format.

Please don’t spend more than 3 hours on your solution. Feel free to allocate that time in a manner that works best for your schedule. You may work in any language you prefer.

We're most interested in both the technical aspects of how you deal with very large integers and how you go about solving the problem. At the onsite, be prepared to discuss your solution, including the choices and tradeoffs you made. Though not required, you are welcome to bring a laptop with you to your interview to walk us through the code.

In [12]:
from collections import defaultdict

In [13]:
def game_of_life(life_106_string: str, iterations: int = 10) -> str:
    """
    Runs Conway's Game of Life for a given number of iterations.

    This implementation is optimized for sparse boards and conceptually infinite grids
    by only tracking the coordinates of live cells.

    Args:
        life_106_string: A string representing the initial state of the board
                         in Life 1.06 format.
        iterations: The number of generations to simulate. Defaults to 10.

    Returns:
        A string representing the final state of the board in Life 1.06 format.
    """

    # --- 1. Parse Input ---
    lines = life_106_string.strip().splitlines()
    if not lines or lines[0] != "#Life 1.06":
        raise ValueError("Invalid Life 1.06 format: Missing or incorrect header.")

    live_cells = set()
    for line in lines[1:]:
        try:
            x, y = map(int, line.split())
            live_cells.add((x, y))
        except (ValueError, IndexError):
            # Skip malformed lines
            continue

    # --- 2. Run Simulation ---
    for _ in range(iterations):
        # defaultdict is efficient for counting neighbors
        neighbor_counts = defaultdict(int)

        # Count neighbors for all relevant cells
        for x, y in live_cells:
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if i == 0 and j == 0:
                        continue
                    neighbor_counts[(x + i, y + j)] += 1

        next_gen_live_cells = set()

        # Apply the rules of life
        for cell, count in neighbor_counts.items():
            # Rule for birth: a dead cell with exactly 3 neighbors comes to life
            if cell not in live_cells and count == 3:
                next_gen_live_cells.add(cell)
            # Rule for survival: a live cell with 2 or 3 neighbors survives
            elif cell in live_cells and (count == 2 or count == 3):
                next_gen_live_cells.add(cell)

        live_cells = next_gen_live_cells

    # --- 3. Format Output ---
    output_lines = ["#Life 1.06"]

    # Sort cells for a consistent, readable output
    sorted_cells = sorted(list(live_cells))

    for x, y in sorted_cells:
        output_lines.append(f"{x} {y}")

    return "\n".join(output_lines)

In [14]:
glider_start = """
#Life 1.06
1 0
2 1
0 2
1 2
2 2
"""

print("--- Glider Initial State ---")
print(glider_start.strip())

# Run the simulation for 10 iterations
glider_end_state = game_of_life(glider_start, 10)

print("\n--- Glider State After 10 Iterations ---")
print(glider_end_state)

Initial Glider string 
#Life 1.06
1 0
2 1
0 2
1 2
2 2

Glider as set of tuples:
 {(1, 2), (2, 1), (0, 2), (2, 2), (1, 0)} 

Glider string after parsing:
 #Life 1.06
1 2
2 1
0 2
2 2
1 0


## Convolution Method

In [16]:
import numpy as np
from scipy.signal import convolve2d

--- Glider Initial State ---
#Life 1.06
1 0
2 1
0 2
1 2
2 2
End Glider as set of tuples:
 {(1, 2), (2, 1), (0, 2), (2, 2), (1, 0)} 


--- Glider State After 10 Iterations ---
{(4, 4), (2, 4), (4, 3), (4, 5), (3, 5)}


In [8]:
def update_board_with_convolution(board: np.ndarray) -> np.ndarray:
    """
    Updates a Game of Life board for one iteration using convolution.
    Assumes board contains 0s for dead cells and 1s for live cells.
    """
    # The kernel sums up the 8 neighbors of each cell.
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])

    # Use convolution to count live neighbors for each cell.
    # 'same' mode ensures the output array has the same shape as the input.
    # 'wrap' handles boundaries by wrapping around (a toroidal array).
    neighbor_count = convolve2d(board, kernel, mode='same', boundary='wrap')

    # Apply the rules of Life using boolean logic on the entire grid at once.

    # Rule 1: A cell is born if it's dead (board == 0) and has 3 neighbors.
    born = (board == 0) & (neighbor_count == 3)

    # Rule 2: A cell survives if it's alive (board == 1) and has 2 or 3 neighbors.
    survives = (board == 1) & ((neighbor_count == 2) | (neighbor_count == 3))

    # Combine the born and surviving cells to get the new board state.
    # The .astype(int) converts the boolean result (True/False) back to 1s and 0s.
    new_board = (born | survives).astype(int)

    return new_board


In [19]:
initial_board = np.zeros((16, 16), dtype=int)
initial_board[2:5, 1:4] = 1 # A horizontal blinker

print("--- Initial Board ---")
print(initial_board)

# Update the board for one step
next_state_board = update_board_with_convolution(initial_board)

print("\n--- Board After 1 Iteration ---")
print(next_state_board) # The blinker should now be vertical

In [18]:
for i in range(10):
    next_state_board = update_board_with_convolution(next_state_board)
    print(f"\n--- Board After {i} Iteration ---")
    print(next_state_board)